# AI Investment Analyst

An agent that:

* Scrapes financial news + quarterly reports

* Runs sentiment analysis

* Combines with stock fundamentals

* Generates investment theses (bullish/bearish)

* Could integrate with Yahoo Finance API for real-time data.

# Imports

In [ ]:
!pip install -U langgraph "langchain[openai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [ ]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16
!pip install -U langchain pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 0.6.4 requires langchain-core>=0.3.67, but you have langchain-core 0.3.33 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 2.1 MB/s eta 0:00:00
  

In [ ]:
!pip install langgraph

In [ ]:
!pip install yfinance


In [ ]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
import uuid
from dotenv import load_dotenv
import os

# Some Tools That My Model Should Use

In [ ]:
f=open("/content/Serper APIkey.txt","r")
S_api_key=f.read()
os.environ["SERPER_API_KEY"] = S_api_key

In [1]:
import yfinance as yf

def ticker_find(ticker_symbol: str):

  "Returns informations about a stock "
  ticker = yf.Ticker(ticker_symbol)
  info = ticker.info
  snapshot = {
        "Company": info.get("longName"),
        "Sector": info.get("sector"),
        "Market Cap": info.get("marketCap"),
        "P/E Ratio": info.get("trailingPE"),
        "EPS": info.get("earningsPerShare"),
        "Dividend Yield": info.get("dividendYield"),
        "52 Week High": info.get("fiftyTwoWeekHigh"),
        "52 Week Low": info.get("fiftyTwoWeekLow"),
    }
  return snapshot

def get_news(ticker:str):
  ticker = yf.Ticker(ticker)
  news = ticker.news
  return news




In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

In [ ]:
from langchain.agents import Tool
#Find the ticker
ticket_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search, like if you need to search the ticker name of a stock.")

ticker_tool=Tool(name='Ticker',
                 func=ticker_find,
                 description="Useful for when you need economic information adout the ticket name of a stock.")

news_tool=Tool(name='News',
                 func=get_news,
                 description="Useful for when you need information about the news of a stock. Use it if you know the ticker of a stock.")

tools=[ticket_search,ticker_tool,news_tool]

In [ ]:
ticket_search.invoke('what is the ticket name of Google?')

"GOOG and GOOGL are stock ticker symbols for Alphabet, Google's parent company. The main difference between the GOOG and GOOGL symbols is that GOOG shares ... Add, edit, or remove your ticket & activity details ; To add: Click Get started. Enter your admission or activity details. Ticket or Activity name and Booking ... A ticket is the digital record of a customer inquiry or support request. When someone emails your support address, your existing system opens up ... Explore cheap flights from United States to anywhere with Google Flights. Find your next flight, track price changes to get the best deals, and book your ... Missing: name | Show results with:name. Google hid its name in the ticket number. 600613 is code for Google. ... Every movie ticket has a serial number, and if Google had its way, every ... If you're having trouble accessing a Google product, there's a chance we're currently experiencing a temporary problem. You can check for outages and downtime ... Usually this means

In [ ]:
ticker_tool.invoke('GOOG')

{'Company': 'Alphabet Inc.',
 'Sector': 'Communication Services',
 'Market Cap': 3038938660864,
 'P/E Ratio': 26.803837,
 'EPS': None,
 'Dividend Yield': 0.33,
 '52 Week High': 253.23,
 '52 Week Low': 142.66}

In [3]:
news=get_news('GOOG')

In [17]:
news_str=[]
for i in range(len(news)):
  news_str.append(news[i]['content']['title'])
  print(news[i]['content']['title'])

Microsoft to invest $30B in AI infrastructure in UK
Google, Webtoon, NYT, Gold: Trending Stocks
Microsoft unveils record £22 billion investment in UK over next four years
UK and US sign $42 bln tech pact to mark Trump's state visit
UK and US agree multibillion-pound big tech deal as Trump arrives for state visit
Why Citizens JMP Sees Profitability Tailwinds for Alphabet (GOOGL)
TikTok Buyers to Include Oracle, Silver Lake and Andreessen
The Smartest Artificial Intelligence (AI) Stocks to Buy With $1,000
Prediction: Oracle Will Surpass Amazon, Microsoft, and Google to Become the Top Cloud for Artificial Intelligence (AI) By 2031
Sundar Pichai has a two-word response to Coinbase CEO Brian Armstrong


In [8]:
news[0]['content']['title']

'Microsoft to invest $30B in AI infrastructure in UK'

# My Models

In [15]:

from huggingface_hub import InferenceClient
f=open("/content/HF API KEY.txt","r")
api_key=f.read()
client = InferenceClient(
    provider="auto",
    api_key=api_key,
)



In [27]:
results=[]
for i in range(len(news_str)):
  result = client.text_classification(
    news_str[i],
    model="ProsusAI/finbert")
  results.append(result[0].label)

In [31]:
results

['neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'neutral']

In [19]:
result = client.text_classification(
    "Bitcoin inches lower after job growth revised down sharply through March: CNBC Crypto World",
    model="ProsusAI/finbert")

In [20]:
result

[TextClassificationOutputElement(label='negative', score=0.9476418495178223),
 TextClassificationOutputElement(label='neutral', score=0.039605021476745605),
 TextClassificationOutputElement(label='positive', score=0.012753166258335114)]

In [ ]:
result[0]

TextClassificationOutputElement(label='negative', score=0.9476418495178223)

In [ ]:
print(result[0].label,result[0].score)

negative 0.9476418495178223


In [ ]:

f=open("/content/OpenAIKey.txt","r")
api_key=f.read()



llm = ChatOpenAI(
    model_name="moonshotai/kimi-k2:free",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.0)

# Let's Create Our Agent

In [ ]:
#State
class State(TypedDict):
  messages: Annotated[List,add_messages]
